# Microsoft 365 Graph による情報取得 (Python)
- MSAL の Device Code Flow でサインイン
- Microsoft Graph のエンドポイント（/me、OneDrive 最近使用ファイル、Teams、チャット、予定表、送信済みメール）を呼び出します

In [ ]:
clientId = "YOUR_CLIENT_ID";
tenantId = "YOUR_TENANT_ID";

In [ ]:
# 依存パッケージをインストール
%pip install msal requests

In [ ]:
import msal
import requests
import json
from pprint import pprint
import datetime

In [ ]:
# テスト
scopes = ["User.Read"]

authority = f"https://login.microsoftonline.com/{tenant_id}"
app = msal.PublicClientApplication(client_id, authority=authority)

# Device Code Flow でトークン取得
flow = app.initiate_device_flow(scopes=scopes)
if "message" in flow:
    print(flow["message"])
else:
    raise ValueError(f"Failed to initiate device flow: {flow}")

result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result["access_token"]
    print("サインインに成功しました。")

    id_claims = result.get("id_token_claims", {})
    preferred_username = id_claims.get("preferred_username") or id_claims.get("upn")
    if preferred_username:
        print(f"ユーザー: {preferred_username}")
    expires_in = result.get("expires_in")
    if expires_in:
        exp_at = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(seconds=expires_in)
        print(f"アクセストークンの有効期限（UTC）: {exp_at.strftime('%Y-%m-%dT%H:%M:%SZ')}")
else:
    print("サインインに失敗しました。")
    pprint(result)

In [ ]:
# スコープ
scopes = [
    "Files.Read",
    "User.Read",
    "Mail.Read",
    "Calendars.Read",
    "Chat.Read",
    "Channel.ReadBasic.All",
    "ChannelMessage.Read.All",
    "Team.ReadBasic.All",
]

authority = f"https://login.microsoftonline.com/{tenant_id}"
app = msal.PublicClientApplication(client_id, authority=authority)

# Device Code Flow でトークン取得
flow = app.initiate_device_flow(scopes=scopes)
if "message" in flow:
    print(flow["message"])
else:
    raise ValueError(f"Failed to initiate device flow: {flow}")

result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result["access_token"]
    print("サインインに成功しました。")

    id_claims = result.get("id_token_claims", {})
    preferred_username = id_claims.get("preferred_username") or id_claims.get("upn")
    if preferred_username:
        print(f"ユーザー: {preferred_username}")
    expires_in = result.get("expires_in")
    if expires_in:
        exp_at = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(seconds=expires_in)
        print(f"アクセストークンの有効期限（UTC）: {exp_at.strftime('%Y-%m-%dT%H:%M:%SZ')}")
else:
    print("サインインに失敗しました。")
    pprint(result)

In [ ]:
# セッション作成
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {access_token}"})

In [ ]:
# OneDrive 最近使用ファイル
def get_recent_files(session):
    r = session.get("https://graph.microsoft.com/v1.0/me/drive/recent")
    r.raise_for_status()
    data = r.json()
    print("[OneDrive Recent Files]")
    for item in data.get('value', []):
        print(f"{item.get('name')} - {item.get('webUrl')}")

# 実行
get_recent_files(session)

In [ ]:
# Teams: チーム、チャネル、メッセージ
def get_teams_channel_messages(session):
    r = session.get("https://graph.microsoft.com/v1.0/me/joinedTeams")
    r.raise_for_status()
    teams = r.json().get('value', [])
    for team in teams:
        team_id = team.get('id')
        team_name = team.get('displayName')
        print(f"[Team: {team_name}]")
        r_ch = session.get(f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels")
        if r_ch.status_code == 404:
            print("  チャネル情報が取得できません（権限不足または API 未対応）")
            continue
        r_ch.raise_for_status()
        channels = r_ch.json().get('value', [])
        for ch in channels:
            channel_id = ch.get('id')
            channel_name = ch.get('displayName')
            print(f" Channel: {channel_name}")
            r_msgs = session.get(f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels/{channel_id}/messages")
            if r_msgs.status_code == 404:
                print("  メッセージ取得エンドポイントが利用できません（権限不足の可能性）")
                continue
            r_msgs.raise_for_status()
            messages = r_msgs.json().get('value', [])
            for m in messages:
                body = m.get('body', {}).get('content')
                print(f"  Message: {body}")
                replies = m.get('replies') or []
                for reply in replies:
                    rbody = reply.get('body', {}).get('content')
                    print(f"    Reply: {rbody}")

# 実行
try:
    get_teams_channel_messages(session)
except Exception as e:
    print("Teams チャネル/メッセージ取得中にエラー:", e)

In [ ]:
# Teams 最近のチャット
def get_recent_chats(session):
    r = session.get("https://graph.microsoft.com/v1.0/me/chats")
    r.raise_for_status()
    data = r.json()
    print("[Teams Recent Chats]")
    for chat in data.get('value', []):
        chat_id = chat.get('id')
        topic = chat.get('topic') or '(No topic)'
        print(f"Chat: {topic} (ID: {chat_id})")

# 実行
get_recent_chats(session)

In [ ]:
# Outlook 予定表
def get_outlook_events(session):
    r = session.get("https://graph.microsoft.com/v1.0/me/events")
    r.raise_for_status()
    data = r.json()
    print("[Outlook Events]")
    for ev in data.get('value', []):
        subj = ev.get('subject')
        start = ev.get('start', {}).get('dateTime')
        end = ev.get('end', {}).get('dateTime')
        print(f"{subj}: {start} - {end}")

# 実行
get_outlook_events(session)

In [ ]:
# Outlook 送信済みメール
def get_outlook_sent_mails(session):
    r = session.get("https://graph.microsoft.com/v1.0/me/mailFolders/sentitems/messages")
    r.raise_for_status()
    data = r.json()
    print("[Outlook Sent Mails]")
    for mail in data.get('value', []):
        subj = mail.get('subject')
        to_recipients = [rcp.get('emailAddress', {}).get('address') for rcp in mail.get('toRecipients', [])]
        print(f"To: {', '.join(filter(None, to_recipients))} | Subject: {subj}")

# 実行
get_outlook_sent_mails(session)